In [165]:
from utils import *

In [244]:
df = pd.read_csv("breast_cancer.csv")

In [245]:
df.sample(5)

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
253,recurrence-events,60-69,ge40,50-54,0-2,no,3,right,left_up,no
230,recurrence-events,50-59,premeno,50-54,9-11,yes,2,right,left_up,no
274,recurrence-events,60-69,ge40,30-34,3-5,yes,2,left,central,yes
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
7,no-recurrence-events,60-69,ge40,20-24,0-2,no,1,left,left_low,no


In [246]:
df.shape

(286, 10)

In [247]:
print("class : ", np.unique(df['class'].values))
print("age : ", np.unique(df['age'].values))
print("menopause : ", np.unique(df['menopause'].values))
print("tumor-size : ", np.unique(df['tumor-size'].values))
print("inv-nodes : ", np.unique(df['inv-nodes'].values))
print("node-caps : ", np.unique(df['node-caps'].values))
print("deg-malig : ", np.unique(df['deg-malig'].values))
print("breast : ", np.unique(df['breast'].values))
print("breast-quad : ", np.unique(df['breast-quad'].values))
print("irradiat : ", np.unique(df['irradiat'].values))

class :  ['no-recurrence-events' 'recurrence-events']
age :  ['20-29' '30-39' '40-49' '50-59' '60-69' '70-79']
menopause :  ['ge40' 'lt40' 'premeno']
tumor-size :  ['0-4' '10-14' '15-19' '20-24' '25-29' '30-34' '35-39' '40-44' '45-49'
 '5-9' '50-54']
inv-nodes :  ['0-2' '12-14' '15-17' '24-26' '3-5' '6-8' '9-11']
node-caps :  ['?' 'no' 'yes']
deg-malig :  [1 2 3]
breast :  ['left' 'right']
breast-quad :  ['?' 'central' 'left_low' 'left_up' 'right_low' 'right_up']
irradiat :  ['no' 'yes']


In [248]:
df = df[(df['node-caps'] != '?') & (df['breast-quad'] != '?')]
df.shape

(277, 10)

In [249]:
df.loc[df['class'] == 'recurrence-events','class'] = 1
df.loc[df['class'] == 'no-recurrence-events','class'] = 2

In [252]:
df.loc[df['age'] == '20-29','age'] = 1
df.loc[df['age'] == '30-39','age'] = 2
df.loc[df['age'] == '40-49','age'] = 3
df.loc[df['age'] == '50-59','age'] = 4
df.loc[df['age'] == '60-69','age'] = 5
df.loc[df['age'] == '70-79','age'] = 6

In [253]:
df.loc[df['menopause'] == 'lt40','menopause'] = 1
df.loc[df['menopause'] == 'ge40','menopause'] = 2
df.loc[df['menopause'] == 'premeno','menopause'] = 3

In [254]:
tumorsize_values = np.array(['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54'])

for i in range(len(tumorsize_values)):
    tumorsize = tumorsize_values[i]
    df.loc[df['tumor-size'] == tumorsize, 'tumor-size'] = i+1

In [255]:
invnodes_values = np.array(['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '24-26'])

for i in range(len(invnodes_values)):
    invnode = invnodes_values[i]
    df.loc[df['inv-nodes'] == invnode, 'inv-nodes'] = i+1

In [256]:
df.loc[df['node-caps'] == 'no','node-caps'] = 0
df.loc[df['node-caps'] == 'yes','node-caps'] = 1

In [257]:
df.loc[df['irradiat'] == 'no','irradiat'] = 0
df.loc[df['irradiat'] == 'yes','irradiat'] = 1

In [258]:
df.drop(['breast', 'breast-quad'], axis=1, inplace=True) # keep ordinal values only
df.sample(5)

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,irradiat
181,2,3,3,8,1,1,3,1
197,2,3,3,3,1,0,2,1
49,2,3,3,5,1,0,1,0
255,1,3,3,7,3,1,3,0
39,2,3,3,7,1,0,1,0


In [265]:
vectors = df.values

In [269]:
vectors.shape

(277, 8)

In [271]:
vectors[:,1:]

array([[2, 3, 7, ..., 0, 3, 0],
       [3, 3, 5, ..., 0, 2, 0],
       [3, 3, 5, ..., 0, 2, 0],
       ...,
       [5, 2, 5, ..., 0, 1, 0],
       [3, 2, 7, ..., 0, 3, 0],
       [4, 2, 7, ..., 0, 3, 0]])

In [272]:
dataset = LabeledSet(7)
dataset.addExamples(vectors[:,1:], vectors[:,0:1])

In [279]:
np.unique(dataset.y)

array([1, 2])

In [280]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

In [283]:
train_set, test_set = split_dataset(dataset, 80)

In [285]:
test_set.size()

57

In [286]:
tree1 = RDMT(rsdm, "shannon", 0, 100, 0.01, [1, 2])
tree1.train(train_set)
tree1.accuracy(test_set)

ValueError: min() arg is an empty sequence